In [1]:
import numpy as np
import tensorflow as tf

tf.enable_eager_execution()
print(tf.__version__)

1.15.0


In [0]:
users = ['Ryan', 'Danielle',  'Vijay', 'Chris']
movies = ['Star Wars', 'The Dark Knight', 'Shrek', 'The Incredibles', 'Bleu', 'Memento']
features = ['Action', 'Sci-Fi', 'Comedy', 'Cartoon', 'Drama']

num_users = len(users)
num_movies = len(movies)
num_feats = len(features)
num_recommendations = 2

**Initialize our users, movie ratings and features**

In [0]:
# each row represents a user's rating for the different movies
users_movies = tf.constant([
                [4,  6,  8,  0, 0, 0],
                [0,  0, 10,  0, 8, 3],
                [0,  6,  0,  0, 3, 7],
                [10, 9,  0,  5, 0, 2]],dtype=tf.float32)

# features of the movies one-hot encoded
# e.g. columns could represent ['Action', 'Sci-Fi', 'Comedy', 'Cartoon', 'Drama']
movies_feats = tf.constant([
                [1, 1, 0, 0, 1],
                [1, 1, 0, 0, 0],
                [0, 0, 1, 1, 0],
                [1, 0, 1, 1, 0],
                [0, 0, 0, 0, 1],
                [1, 0, 0, 0, 1]],dtype=tf.float32)

**Computing the user feature matrix**

In [8]:
weighted_feature_matrices = [tf.expand_dims(tf.transpose(users_movies)[:,i],axis=1) * movies_feats for i in range(num_users)]
users_movies_feats = tf.stack(weighted_feature_matrices)
users_movies_feats

<tf.Tensor: id=116, shape=(4, 6, 5), dtype=float32, numpy=
array([[[ 4.,  4.,  0.,  0.,  4.],
        [ 6.,  6.,  0.,  0.,  0.],
        [ 0.,  0.,  8.,  8.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.]],

       [[ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0., 10., 10.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  8.],
        [ 3.,  0.,  0.,  0.,  3.]],

       [[ 0.,  0.,  0.,  0.,  0.],
        [ 6.,  6.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  3.],
        [ 7.,  0.,  0.,  0.,  7.]],

       [[10., 10.,  0.,  0., 10.],
        [ 9.,  9.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 5.,  0.,  5.,  5.,  0.],
        [ 0.,  0.,  0.,  0.,  0.],
        [ 2.,  0.,  0.,  0.,  2.]]], dtype=float32)>

In [0]:
users_movies_feats_sums = tf.reduce_sum(users_movies_feats,axis = 1)
users_movies_feats_totals = tf.reduce_sum(users_movies_feats_sums,axis = 1)

In [13]:
users_feats = tf.stack([users_movies_feats_sums[i,:]/users_movies_feats_totals[i] for i in range(num_users)])
users_feats 

<tf.Tensor: id=199, shape=(4, 5), dtype=float32, numpy=
array([[0.25      , 0.25      , 0.2       , 0.2       , 0.1       ],
       [0.0882353 , 0.        , 0.29411766, 0.29411766, 0.32352942],
       [0.44827586, 0.20689656, 0.        , 0.        , 0.3448276 ],
       [0.3880597 , 0.2835821 , 0.07462686, 0.07462686, 0.17910448]],
      dtype=float32)>

**Ranking feature relevance for each user**

In [16]:
top_users_features = tf.nn.top_k(users_feats, num_feats)[1]

for i in range(num_users):
    feature_names = [features[int(index)] for index in top_users_features[i]]
    print('{}: {}'.format(users[i],feature_names))

Ryan: ['Action', 'Sci-Fi', 'Comedy', 'Cartoon', 'Drama']
Danielle: ['Drama', 'Comedy', 'Cartoon', 'Action', 'Sci-Fi']
Vijay: ['Action', 'Drama', 'Sci-Fi', 'Comedy', 'Cartoon']
Chris: ['Action', 'Sci-Fi', 'Drama', 'Comedy', 'Cartoon']


**Determining movie recommendations**

In [17]:
users_ratings = tf.matmul(users_feats,tf.transpose(movies_feats))
users_ratings

<tf.Tensor: id=304, shape=(4, 6), dtype=float32, numpy=
array([[0.6       , 0.5       , 0.4       , 0.65      , 0.1       ,
        0.35      ],
       [0.4117647 , 0.0882353 , 0.5882353 , 0.67647064, 0.32352942,
        0.4117647 ],
       [1.        , 0.6551724 , 0.        , 0.44827586, 0.3448276 ,
        0.79310346],
       [0.8507463 , 0.6716418 , 0.14925373, 0.53731346, 0.17910448,
        0.5671642 ]], dtype=float32)>

In [22]:
users_ratings_new = tf.where(tf.equal(users_movies, tf.zeros_like(users_movies)),
                                  users_ratings,
                                  tf.zeros_like(tf.cast(users_movies, tf.float32)))
users_ratings_new

<tf.Tensor: id=316, shape=(4, 6), dtype=float32, numpy=
array([[0.        , 0.        , 0.        , 0.65      , 0.1       ,
        0.35      ],
       [0.4117647 , 0.0882353 , 0.        , 0.67647064, 0.        ,
        0.        ],
       [1.        , 0.        , 0.        , 0.44827586, 0.        ,
        0.        ],
       [0.        , 0.        , 0.14925373, 0.        , 0.17910448,
        0.        ]], dtype=float32)>

In [25]:
top_movies = tf.nn.top_k(users_ratings_new, num_recommendations)[1]

for i in range(num_users):
    movie_names = [movies[index] for index in top_movies[i]]
    print('{}: {}'.format(users[i],movie_names))

Ryan: ['The Incredibles', 'Memento']
Danielle: ['The Incredibles', 'Star Wars']
Vijay: ['Star Wars', 'The Incredibles']
Chris: ['Bleu', 'Shrek']
